In [ ]:
import pandas, re, datetime

# Read data

In [ ]:
def toInt(s, d = 1):
    try:
        s = str(int(s))
        return addZeros(s, d)
    except:
        return "X"
def addZeros(s, d):
    s = str(s)
    return '0' * (d - len(str(s))) + s 

def standardiseVenezuelan(s):
    s = str(s)
    s = s.strip()
    s = s.lower().replace('ven', 'v')
    s = s.replace('v', 'VEN')
    return s

def addID01(df):
    placeId = '806001061-8'
    placeId = addZeros(placeId, 20)
    df['ID01'] = placeId + df['VAR_0019'].map(lambda s: addZeros(s,20))  \
                                              + df['VAR_0040'].map(lambda s: toInt(s, 2)) + \
                                            df['VAR_0286'].map(toInt)
    

In [ ]:
nivelN1 = pandas.read_csv('Data/nivelN1_manual.csv')
nivelN8 = pandas.read_csv('Data/nivelN8_manual.csv')
nivelN8.index = nivelN8.ID01
nivelN1.index = nivelN1.ID01
nivelN1 = nivelN1.loc[~nivelN1.index.duplicated(keep='first')]
nivelN8 = nivelN8.loc[~nivelN8.index.duplicated(keep='first')]

nivelN1 = nivelN1.join(nivelN8, rsuffix = 'r')

nivelN1['VAR_0019'] = nivelN1['VAR_0019'].map(standardiseVenezuelan)
addID01(nivelN1)
nivelN1 = nivelN1.loc[~nivelN1.index.duplicated(keep='first')]
nivelN1['VAR_0183'] = nivelN1.VAR_0183.map(lambda s: datetime.datetime.strptime(s , '%m/%d/%y %H:%M:%S') if s == s else datetime.datetime(1,1,1))

automatic = pandas.read_csv('Data/results1.csv')
automatic['VAR_0019'] = automatic['VAR_0019'].map(standardiseVenezuelan)
addID01(automatic)
automatic.index = automatic.ID01
automatic['VAR_0183'] = automatic.VAR_0183.map(lambda s: datetime.datetime.strptime(s , '%Y-%m-%d %H:%M:%S'))

In [ ]:
pathFolder = 'Casos2019'
casos = pandas.read_csv(pathFolder + '/casos.csv', index_col = 0)
pacientes = pandas.read_csv(pathFolder + '/pacientes.csv', index_col = 0)
pacientes.index = pacientes.index.map(str)

registros = pandas.read_csv(pathFolder + '/registros.csv', index_col = 0)
registros.index = registros.index.map(str)

#diagnosis = pandas.read_csv('Venezolanas2/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv(pathFolder + '/procedimientos.csv', index_col = 0)
procedimientos.index = procedimientos.index.map(str)
registrosByCaso = registros.groupby('Caso')

entriesInfirmery = pandas.read_csv(pathFolder + '/enfermeriaMedidas.csv', index_col = 0)
entriesInfirmeryByCase = entriesInfirmery.groupby('IdAdmision')

# Explore differences

- Explore differences in the values, and which values I could not recover
- Some of the registers are missing, but the matching can still be done (it is in database transformer). There are ~ 90 missing records that I could not find.

In [ ]:
import datetime
import caseTransformer


In [ ]:
c = 0
errors = []
automatic2manual = {}
for k, i in nivelN1.iterrows():
    if '2019' not in str(i.VAR_0183):
        continue
    
    if i.ID01 not in automatic.index:
        c += 1
        errors.append((k, i.VAR_0019, i.VAR_0183))
        j = i
    else:
        automatic2manual[i.ID01] = k
print (len(errors), len(automatic2manual))

In [ ]:
import difflib
def similarTexts(s1, s2, th = .8):
    #s1 = s1.replace('VEN', '').strip()
    #s2 = s2.replace('VEN', '').strip()

    d =difflib.SequenceMatcher(a = s1, b = s2)
    r = sum((i.size for i in d.get_matching_blocks())) / min(len(s1), len(s2))
    return r > th

def nearOperation(pId, dayOperation, automatic, pIDOut):
    registriesNear = automatic.loc[((dayOperation - datetime.timedelta(days = 5)) < automatic.VAR_0183) & 
            ((dayOperation + datetime.timedelta(days = 5)) > automatic.VAR_0183)]
    similar = registriesNear.VAR_0019.map(lambda s:  similarTexts(s, pId))
    if sum(similar) == 1:
        pIDOut['res'] = registriesNear[similar].ID01.iloc[0]
        return True

    else:
        #print('Error')
        return False


# Check whether there exists a case with 
cF, cT = 0, 0 
for i,(iD01, pId, dayOperation) in enumerate(errors):
    pIDOut = {}
    if pId in automatic.VAR_0019.values:
        cT += 1
        id01Automatic = automatic.loc[automatic.VAR_0019 == pId, 'ID01'].iloc[0]
        automatic2manual[id01Automatic] = iD01
    elif 'VEN' + pId in automatic.VAR_0019.values:
        cT += 1
        id01Automatic = automatic.loc[automatic.VAR_0019 == 'VEN' + pId, 'ID01'].iloc[0]
        automatic2manual[id01Automatic] = iD01

    elif nearOperation(pId, dayOperation, automatic, pIDOut):
        automatic2manual[pIDOut['res']] = iD01
       
    #elif dayOperation <= datetime.datetime(2019, 1, 20):
    #    pass
    else:
        print(pId)
        cF += 1
    
print(cF, cT)

In [ ]:
manualFiltered= nivelN1.loc[list(automatic2manual.values())]
automaticFiltered = automatic.loc[list(automatic2manual.keys())]

In [ ]:
#Normalize the  dates so that they can be compared
import caseTransformer
def parseDateManual(s):
    if s != s or '-' in s:
        return s
    d = datetime.datetime.strptime(s, '%m/%d/%y %H:%M:%S')
    return ('%d-%02d-%02d' % (d.year,d.month, d.day))

for d in caseTransformer.dateVars:
    manualFiltered[d] = manualFiltered[d].map(parseDateManual)

In [ ]:
dfNamesVars = pandas.read_excel('parsingData/variables_sip.xlsx')
renameDict = {k : (k + ' = ' + v) for  k, v in zip(dfNamesVars['Unnamed: 2'].values, dfNamesVars['Unnamed: 3'].values)}
renameDict['VAR_0189'] = 'Corticoides / semana de inicio'
groupedBy = dfNamesVars.groupby('Unnamed: 2')
def renameDatabaseSIP(df):
    df.columns = df.columns.map(lambda s: s if s not in renameDict.values() else s + 'Self')
    df.columns = df.columns.map(lambda s: s if s not in renameDict else renameDict[s])
    for g in groupedBy.groups:
        if len(groupedBy.get_group(g)) == 1 or renameDict[g] not in df:
            continue
        colname = renameDict[g]
        for (_, r) in groupedBy.get_group(g)[['Unnamed: 4', 'Unnamed: 5']].iterrows():
            option = r['Unnamed: 4']
            value = r['Unnamed: 5']
            df.loc[df[colname] == option, colname] = value
renameDatabaseSIP(manualFiltered)
renameDatabaseSIP(automaticFiltered)

In [ ]:
manualFiltered.columns

In [ ]:
commonCols = set(manualFiltered.columns).intersection(automaticFiltered.columns)
print('Common columns:', len(commonCols), len(manualIFiltered.columns))

In [ ]:
import collections
errorCount = collections.defaultdict(lambda : 0)

In [ ]:
pacientes['Identificacion'].values

In [ ]:
'V27180231' in pacientes['Identificacion'].values

In [ ]:
for i, (kAuto, kManual) in enumerate(automatic2manual.items()):
    print('Differences :', kAuto, kManual)
    for c in commonCols:
        if automaticFiltered.loc[kAuto, c] != manualFiltered.loc[kManual, c]:
            print('VAR %s : auto= %s  , manual= %s' % (c, automaticFiltered.loc[kAuto, c], manualFiltered.loc[kManual, c]))
            errorCount[c] +=1
    #print('====\n\n')
    #for c in filter(lambda c: c not in commonCols, manualFiltered.columns):
    #    print('VAR %s : manual= %s' % (c,  manualFiltered.loc[kManual, c]))
    #    
    #print('====\n\n')
    #for c in filter(lambda c: c not in commonCols and c.startswith('VAR'), automaticFiltered.columns):
    #    print('VAR %s : manual= %s' % (c,  automaticFiltered.loc[kAuto, c]))


In [ ]:
def hclin2pid(hClin):
    return pacientes.loc[pacientes.Identificacion == hClin].index[0]

In [ ]:
automatic[automatic.VAR_0019 == errors[25]]

In [ ]:
id = hclin2pid(errors[30])
registros.loc[registros.NumeroHistoria == int(id)]

In [ ]:
casos.loc['AD326269']

In [ ]:
id = hclin2pid('1050005797')
casos.loc[casos.Paciente == id]

In [ ]:
j.VAR_0183

In [ ]:
for _, i in automatic.iterrows():
    if '1050005797' in i.ID01:
        print(i.ID01)

In [ ]:
automatic.VAR_0183